In [19]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
#from tqdm.autonotebook import tqdm # if first is not working
import albumentations as A
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import timm

# for hubert-base-cc
from transformers import AutoModel, AutoTokenizer

# for sentiment-hts2-hubert-hungarian
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# for xlm-roberta-base
from transformers import AutoTokenizer, AutoModelForMaskedLM

import pprint

In [20]:
#!pip install kaggle --upgrade
#os.environ['KAGGLE_USERNAME'] = "oroszerik"
#os.environ['KAGGLE_KEY'] = "abd8ef0f5dd4ed1f95df201400bb3952"

In [21]:
# you can select your dataset here
# for flickr-8k img dataset
# unzip the images to the "images_8k" folder
#!kaggle datasets download -d adityajn105/flickr8k
##!unzip flickr8k.zip
dataset = "8k"

# for flickr-30k img dataset
# it downloads a zip, if you unzip it you get the dataset twice 
# unzip the images to the "images_30k" folder
#!kaggle datasets download -d hsankesara/flickr-image-dataset
##!unzip flickr-image-dataset.zip
#dataset = "30k"

In [22]:
#!nvidia-smi

In [23]:
# check if GPU available
print(torch.cuda.is_available())
# get index of currently selected device
#print(torch.cuda.current_device())
# get number of GPUs available
#print(torch.cuda.device_count())
# get the name of the device
#print(torch.cuda.get_device_name(0))

False


In [24]:
# we translated these two caption text files
if dataset == "8k":
  image_path = "images_8k"
  captions_path = "captions"
  file="captions_8k_hu"
  csv_name = f"{file}.csv"
  df = pd.read_csv(f"{captions_path}/{file}.txt", delimiter="|")
  df.columns = ['image', 'caption_number', 'caption']
  df['caption'] = df['caption']
  df['caption_number'] = df['caption_number']
  ids = [id_ for id_ in range(len(df) // 5) for i in range(5)]
  df['id'] = ids
  df.to_csv(f"{captions_path}/{csv_name}", index=False) 
elif dataset == "30k":
  image_path = "images_30k"
  captions_path = "captions"
  file="captions_30k_hu"
  csv_name = f"{file}.csv"
  df = pd.read_csv(f"{captions_path}/{file}.txt", delimiter="|")
  df.columns = ['image', 'caption_number', 'caption']
  df['caption'] = df['caption']
  df['caption_number'] = df['caption_number']
  ids = [id_ for id_ in range(len(df) // 5) for i in range(5)]
  df['id'] = ids
  df.to_csv(f"{captions_path}/{csv_name}", index=False)
df

,image,caption_number,caption,id
0,1000268201_693b08cb0e.jpg,0,Egy rózsaszín ruhás gyerek felmászik egy lépcs...,0
1,1000268201_693b08cb0e.jpg,1,Egy lány bemegy egy faépületbe.,0
2,1000268201_693b08cb0e.jpg,2,Egy kislány bemászik egy fából készült játszóh...,0
3,1000268201_693b08cb0e.jpg,3,Egy kislány felmászik a lépcsőn a játszóházához.,0
4,1000268201_693b08cb0e.jpg,4,Egy rózsaszín ruhás kislány bemegy egy fakabinba.,0
...,...,...,...,...
40450,997722733_0cb5439472.jpg,0,Egy rózsaszín inges férfi felmászik egy szikla...,8090
40451,997722733_0cb5439472.jpg,1,Egy férfi sziklát mászik a magasban a levegőben.,8090
40452,997722733_0cb5439472.jpg,2,Egy piros inges személy felkapaszkodik egy seg...,8090
40453,997722733_0cb5439472.jpg,3,Egy sziklamászó piros ingben.,8090


In [25]:
class CFG:
    debug = False
    image_path = image_path
    captions_path = captions_path
    batch_size = 32
    num_workers = 0 # it was 4 on colab, should be 0 on our pc-s
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1 # for server running it could be 10, 30 or even 50
    factor = 0.8
    epochs = 1  # could be 1000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    model_name = 'tf_efficientnetv2_b1'
    image_embedding = 1280  # for efficientnetv2b1

    text_model_name = ["SZTAKI-HLT/hubert-base-cc", "xlm-roberta-base", "NYTK/sentiment-hts2-hubert-hungarian"]
    text_encoder_model = text_model_name[2]
    text_embedding = 768
    text_tokenizer = text_encoder_model
    max_length = 200

    pretrained = True   # for both image encoder and text encoder
    trainable = False   # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 240  # for efficientnetv2b1

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256 
    dropout = 0.1

cpu


In [26]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]


In [27]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names 
        """

        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        #item['caption'] = self.captions[idx]   # it is not needed, just to help visualize

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

In [28]:
# available pretrained img models
pp = pprint.PrettyPrinter(indent=4)
avail_pretrained_models = timm.list_models(pretrained=True)
#pp.pprint(avail_pretrained_models)
#len(avail_pretrained_models)

In [29]:
class ImageEncoder(nn.Module):
    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name,
            pretrained, 
            num_classes=0,
            global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [30]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()

        print(model_name)
        if(CFG.text_model_name.index(model_name)==0):
            self.model = AutoModel.from_pretrained(model_name) # hubert-base-cc
        elif(CFG.text_model_name.index(model_name)==1):
            self.model = AutoModelForMaskedLM.from_pretrained(model_name)  # xlm-roberta-base
        elif(CFG.text_model_name.index(model_name)==2):
            self.model = AutoModelForSequenceClassification.from_pretrained(model_name) # sentiment-hts2-hubert-hungarian
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):

        if(CFG.text_model_name.index(CFG.text_encoder_model)==0):
            output = self.model(input_ids=input_ids, attention_mask=attention_mask)    # hubert-base-cc
            last_hidden_state = output.last_hidden_state   # hubert-base-cc
        else:
            output = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)    # other
            last_hidden_state = output.hidden_states[-1]   # other
        return last_hidden_state[:, self.target_token_idx, :]

In [31]:
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)
    
    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [32]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [33]:
def make_train_valid_dfs():
    dataframe = pd.read_csv(f"{CFG.captions_path}/{csv_name}")
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    np.random.shuffle(image_ids)
    #valid_ids = np.random.choice(image_ids, size=int(0.2 * len(image_ids)), replace=False)
    #train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    test_ids = image_ids[int(len(image_ids)*0.8):]
    valid_ids = image_ids[int(len(image_ids)*0.6):int(len(image_ids)*0.8)]
    train_ids = image_ids[:int(len(image_ids)*0.6)]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    test_dataframe = dataframe[dataframe["id"].isin(test_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe, test_dataframe


def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=CFG.batch_size,num_workers=CFG.num_workers,shuffle=True if mode == "train" else False,)
    return dataloader

In [34]:
a=np.array([1,3,5,6,734,2,763,13,543,12])
np.random.seed(10)
print(np.random.choice(a,size=5,replace=False))
print(np.random.choice(a,size=5,replace=False))

[543   5   2 763   6]
[  2   6 734  13 763]


In [35]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter


def valid_epoch(model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter


def main():
    train_df, valid_df, test_df = make_train_valid_dfs()

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)

    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")

    model = CLIPModel().to(CFG.device)
    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(
            model.image_projection.parameters(), model.text_projection.parameters()
        ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=CFG.patience, factor=CFG.factor
    )
    step = "epoch"

    best_loss = float('inf')
    for epoch in range(CFG.epochs):
        print(f"Epoch: {epoch + 1}")
        model.train()
        train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        model.eval()
        with torch.no_grad():
            valid_loss = valid_epoch(model, valid_loader)
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), "best_model.pt")
            print("This was the best model yet, model is saved.")
        
        lr_scheduler.step(valid_loss.avg)

In [36]:
main()

NYTK/sentiment-hts2-hubert-hungarian
Epoch: 1


  1%|▏         | 11/759 [00:45<50:01,  4.01s/it, lr=0.0001, train_loss=28.5]

In [ ]:
def get_image_embeddings(valid_df, model_path):

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)

    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")
    
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()
    
    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

In [ ]:
_, valid_df, test_df = make_train_valid_dfs()
model, image_embeddings = get_image_embeddings(test_df, "best_model.pt") # it was valid_df

In [ ]:
from cgitb import text
from math import radians


def find_matches(model, image_embeddings, query, image_filenames, n=9):
    
    tokenizer = AutoTokenizer.from_pretrained(CFG.text_tokenizer)

    encoded_query = tokenizer([query])
    batch = {
        key: torch.tensor(values).to(CFG.device)
        for key, values in encoded_query.items()
    }
    with torch.no_grad():
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        text_embeddings = model.text_projection(text_features)
    #print('text_embeddings.size()',text_embeddings.size())  # contains the embedded text from our query
    #print(text_embeddings)  # is this the starting point (input) for our diff model?
    
    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    dot_similarity = text_embeddings_n @ image_embeddings_n.T
    #print(image_embeddings_n.size())    # contains len=256 vectors for each x images (x depends on the dataset)
    values, indices = torch.topk(dot_similarity.squeeze(0), n * 5)
    #print(indices.size())   # best n*5
    #print(values.size())    # best n*5
    #print(indices[0:5])  # (best) indexes
    #print(values[0:5])   # (best) similarity values
    #print('image_embeddings[indices[0]]',image_embeddings[indices[0]])  # i could get the best matching image's embeddings this way
    best_img_emb_n = F.normalize(image_embeddings[indices[0]], p=2, dim=-1)
    #dot_sim_query_img =text_embeddings_n @ best_img_emb_n.T
    #print('dot_sim_query_img',dot_sim_query_img)    
    matches = [image_filenames[idx] for idx in indices[::5]]
    
    _, axes = plt.subplots(3, 3, figsize=(10, 10))
    for match, ax in zip(matches, axes.flatten()):
        image = cv2.imread(f"{CFG.image_path}/{match}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis("off")
    
    plt.show()

In [ ]:
find_matches(model, 
             image_embeddings,
             query="ló",
             image_filenames=valid_df['image'].values,
             n=9)